In [ ]:
!pip install reportlab pypdf

import io
import os
from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from pypdf import PdfReader, PdfWriter
from google.colab import files
import time

# --- PDF Overlay Function ---
def create_invoice_overlay(data):
    packet = io.BytesIO()
    c = canvas.Canvas(packet, pagesize=A4)
    width, height = A4 # A4 পেজের সাইজ নেওয়া হলো (হিসাবের সুবিধার জন্য)

    # --- Settings ---
    left_margin = 50
    top_y = 660
    line_height = 15

    # Customer Info
    c.setFont("Helvetica-Bold", 12)
    c.drawString(left_margin, top_y, f"Customer Name: {data['name']}")

    c.setFont("Helvetica", 10)
    c.drawString(left_margin, top_y - line_height, f"Address: {data['address']}")
    c.drawString(left_margin, top_y - (line_height * 2), f"Mobile: {data['mobile']}")

    # Date
    c.drawString(400, top_y, f"Date: {data['date']}")

    # Table Header
    table_y = top_y - 60
    c.setFont("Helvetica-Bold", 10)

    col_sl = 50
    col_desc = 90
    col_qty = 300
    col_rate = 360
    col_total = 450

    c.drawString(col_sl, table_y, "SL")
    c.drawString(col_desc, table_y, "Description")
    c.drawString(col_qty, table_y, "Qty")
    c.drawString(col_rate, table_y, "Rate")
    c.drawString(col_total, table_y, "Total Amount")

    c.line(40, table_y - 5, 550, table_y - 5)

    # Items
    c.setFont("Helvetica", 10)
    current_y = table_y - 20
    grand_total = 0

    for item in data['items']:
        c.drawString(col_sl, current_y, str(item['sl']))
        c.drawString(col_desc, current_y, item['desc'])
        c.drawString(col_qty, current_y, str(item['qty']))
        c.drawString(col_rate, current_y, f"{item['rate']:.2f}")

        total = item['qty'] * item['rate']
        grand_total += total
        c.drawString(col_total, current_y, f"{total:.2f}")
        current_y -= 15

    # Total
    c.line(350, current_y + 5, 550, current_y + 5)
    c.setFont("Helvetica-Bold", 11)
    c.drawString(360, current_y - 15, "Grand Total:")
    c.drawString(col_total, current_y - 15, f"{grand_total:.2f}")

    # ==========================================
    # --- FOOTER NOTE / WATERMARK SECTION ---
    # ==========================================
    c.setFont("Helvetica-Oblique", 8)  # Oblique মানে Italic ফন্ট, সাইজ ছোট (8)
    note_text = "**This is a computer generated invoice and does not require a signature. It is officially valid.**"

    # পেজের একদম নিচে মাঝখানে বসানো (width/2 = Center)
    # 40 হলো নিচ থেকে উচ্চতা
    c.drawCentredString(width / 2, 40, note_text)

    c.save()
    packet.seek(0)
    return packet

# === MAIN PROGRAM START ===

print("=== INVOICE GENERATOR SYSTEM ===")
print("Note: শুরুতে একবার আপনার PDF প্যাড আপলোড করতে হবে।\n")

# 1. Upload Pad (Only Once)
pad_filename = ""
uploaded = files.upload()

if not uploaded:
    print("❌ কোনো ফাইল আপলোড করা হয়নি! প্রোগ্রাম বন্ধ করা হলো।")
else:
    pad_filename = list(uploaded.keys())[0]
    print(f"✅ Pad Uploaded: {pad_filename}")

    # 2. Infinite Loop
    while True:
        print("\n" + "="*40)
        print("   NEW INVOICE ENTRY")
        print("="*40)

        # Customer Data Input
        c_name = input("Enter Customer Name: ")
        c_address = input("Enter Address: ")
        c_mobile = input("Enter Mobile: ")
        c_date = input("Enter Date (DD-MM-YYYY) [Enter for Today]: ")

        if not c_date:
            c_date = datetime.now().strftime("%d-%m-%Y")

        # Items Collection
        items = []
        sl = 1
        print("\n--- Add Items ---")

        while True:
            add_choice = input(f"Item #{sl} add korben? (y/n): ").lower()

            if add_choice == 'n':
                break
            elif add_choice == 'y':
                desc = input("  Description: ")
                try:
                    qty = int(input("  Quantity: "))
                    rate = float(input("  Rate: "))
                    items.append({'sl': sl, 'desc': desc, 'qty': qty, 'rate': rate})
                    sl += 1
                except ValueError:
                    print("  ⚠️ Error: Quantity ar Rate number hote hobe!")
            else:
                print("  ⚠️ Please type 'y' or 'n'")

        # Generate Invoice
        if items:
            data = {
                'name': c_name, 'address': c_address,
                'mobile': c_mobile, 'date': c_date, 'items': items
            }

            # Create PDF
            overlay_pdf = create_invoice_overlay(data)
            pad_reader = PdfReader(pad_filename)
            overlay_reader = PdfReader(overlay_pdf)
            writer = PdfWriter()

            page = pad_reader.pages[0]
            page.merge_page(overlay_reader.pages[0])
            writer.add_page(page)

            timestamp = int(time.time())
            clean_name = c_name.replace(" ", "_")
            output_filename = f"Invoice_{clean_name}_{timestamp}.pdf"

            with open(output_filename, "wb") as f:
                writer.write(f)

            print(f"\n✅ Generated: {output_filename}")
            files.download(output_filename)

        else:
            print("\n⚠️ No items added. Skipping.")

        # 3. Ask for Another Invoice
        print("\n" + "-"*30)
        next_action = input("Are you want to generate ANOTHER invoice? (y/n): ").lower()

        if next_action != 'y':
            print("\n👋 Thank you! Exiting program.")
            break
